In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
import pandas as pd
import numpy as np

import pyarrow.parquet as pq

import time
import datetime as dt


from pathlib import Path

#### LOAD DATA:
1. <b>CANARY(TREATED)
2. <b>BIOMETRICS(OUTCOME)
3. <b>PMPM(ELIGIBLE)

In [3]:
%%time
canary_p = '../../data/CANARY/CANARY/PERSON_SUMMARY'
biometrics_p = '../../data/BIOMETRICS'
pmpm_p = '../../temp/PMPM.p'

canary = pq.read_table(canary_p).to_pandas()
biometrics = pq.read_table(biometrics_p).to_pandas()
pmpm = pd.read_pickle(pmpm_p)


CPU times: user 33.9 s, sys: 10.3 s, total: 44.2 s
Wall time: 25.7 s


#### Data Processing and Filtering
1. For Biometric data
2. For PMPM Data
3. For Canary data

#### 1) Biometric data 
1. Pick Category of data
1. Standarize Dates: MYR, Datetime Date format
2. Average Values: In the same Month Year

In [4]:
biometrics.columns

Index(['PERS_ID', 'EVENT_DATE', 'DATE_TYPE', 'BIOMETRIC_CATEGORY',
       'EVENT_CODE', 'CODING_STANDARD', 'SOURCE', 'RESULT_VALUE'],
      dtype='object')

In [5]:
%%time
biometrics.groupby(by = 'BIOMETRIC_CATEGORY').apply(len)

CPU times: user 14.6 s, sys: 2.54 s, total: 17.1 s
Wall time: 17.1 s


BIOMETRIC_CATEGORY
A1C                             740341
BMI                            9607665
BNP                              39357
Blood Pressure (Diastolic)     4640018
Blood Pressure (Systolic)      4934403
Cholesterol (LDL)              1240007
DCSI-EGFR                       592754
DCSI-MICROALBUMIN                77517
DCSI-SERUM                     2576985
Ferritin                        207220
Height                        11179070
Hematocrit                     2401459
Hemoglobin                     2735398
Iron                            185396
NT_proBNP                         8261
PSA                             299144
Potassium                      2490129
Troponin                         87111
Weight                        13305577
eGFR                            861438
dtype: int64

#### Above we see categories we would like to pick for outcomes (Pick BMI for example)

In [6]:
%%time
filt = biometrics.BIOMETRIC_CATEGORY == 'BMI'
biometrics = biometrics[filt].reset_index(drop = True)

#make ids integers, and result values floats
biometrics['PERS_ID'] = biometrics['PERS_ID'].astype(int)
biometrics.RESULT_VALUE = biometrics.RESULT_VALUE.astype(float)

#make dates datetime date
biometrics.EVENT_DATE = pd.to_datetime(biometrics.EVENT_DATE).dt.date

CPU times: user 24.5 s, sys: 1.81 s, total: 26.4 s
Wall time: 26.3 s


#### Make Biometrics EVENT_DATE_MYR and all EVENT_DATE dates to start from day 1

In [7]:
def make_days1( row ):
    """
    takes a datetime.date
    object and makes the day 1
    """
    year = row.year
    month = row.month
    
    return dt.date(year = year, month = month, day = 1)

def make_date_to_MYR( row ):
    """
    takes a date object
    and returns a 6 digit integer
    year-month
    
    """
    
    new_row = row.year*100 +row.month
    
    return new_row

#### Apply Date modifications to Biometrics data

In [8]:
%%time
biometrics.EVENT_DATE = biometrics.EVENT_DATE.apply(make_days1)
biometrics['EVENT_DATE_MYR'] = biometrics.EVENT_DATE.apply(make_date_to_MYR)

CPU times: user 12 s, sys: 448 ms, total: 12.5 s
Wall time: 12.5 s


#### Compute the Mean of biometric values occuring the same date and keep this as the new value, drop the duplicates

In [9]:
%%time
#compute the mean biometric value per month for each patient
bio_mean = biometrics.groupby(by = ['PERS_ID', 'EVENT_DATE_MYR']).RESULT_VALUE.mean()

#drop the duplicate ids-dates tuples and the biometrics results values
biometrics.drop_duplicates(subset = ['PERS_ID', 'EVENT_DATE_MYR'], inplace = True)
biometrics.drop(labels = 'RESULT_VALUE', axis = 1, inplace = True)

#merge the mean result values in the initial biometrics dataset
biometrics = biometrics.merge(bio_mean, on = ['PERS_ID', 'EVENT_DATE_MYR'])

#drop unecessary columns from biometrics
biometrics.drop(labels = ['DATE_TYPE', 'CODING_STANDARD', 'SOURCE', 'EVENT_CODE'], inplace = True, axis = 1)

CPU times: user 3.55 s, sys: 732 ms, total: 4.28 s
Wall time: 4.28 s


In [10]:
biometrics.head(2)

PERS_ID  EVENT_DATE BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE
0  2570719  2019-10-01                BMI          201910     25.545750
1  2570719  2018-12-01                BMI          201812     26.473165

#### Drop Ids in Biometrics that do not have at least two measurements on record in different dates 

In [11]:
%%time
number_of_measurements = biometrics.groupby('PERS_ID').apply(len)
ids_more_than2 = number_of_measurements[number_of_measurements > 1].index

biometrics = biometrics[biometrics.PERS_ID.isin(ids_more_than2)]

CPU times: user 1.44 s, sys: 95.2 ms, total: 1.54 s
Wall time: 1.53 s


In [12]:
biometrics.PERS_ID.unique().shape, ids_more_than2.shape, number_of_measurements.shape

((248608,), (248608,), (306716,))

#### PMPM_PROCESSING
Make ids integers, make gender a binary variable (0,1)

In [13]:
%%time
pmpm.PERS_ID = pmpm.PERS_ID.astype(int)
pmpm.MBR_GNDR = pmpm.MBR_GNDR.replace(['F', 'M'], [0,1])

CPU times: user 1.13 s, sys: 31.7 ms, total: 1.16 s
Wall time: 1.16 s


#### Continuous enrollement Filter

In [14]:
def is_consecutive(date1, date2):
    m1, y1 = date1%100, date1//100
    m2, y2 = date2%100, date2//100
    
    if m2%12 == (m1%12 + 1):
        if (y1 == y2) or (y2 == y1+1):
            return True
        
    if (m1 == 11) and (m2 == 12):
        if (y1 == y2) or (y2 == y1+1):
            return True
        
    
    return False
    
def enrollement_check( data, time_in_months):
    """
    Finds All Patients with Continuous Enrollement
    data in a Nx2 matrix, N is the size of PMPM
    columns: 1:ids, 2:Myr, 
    
    """
    
    tm = time_in_months
    current_id = data[0,0]
    counter = 1
    current_date = data[0,1]
    start_date = current_date
    N = data.shape[0]
    enhanced_data = []
    
    #for all dates and ids in dataset
    for i in range(1,N):
        idi = data[i, 0]
        date_i = data[i, 1]
        
        if idi == current_id: #we still checking a specific patient
            if is_consecutive(current_date, date_i):
                counter += 1
            else:
                if counter >= tm:
                    enhanced_data.append([current_id, start_date, current_date, counter])
                
                counter = 1
                start_date = date_i
        
        else: #change id
            if counter >= tm: #if previous id was satisfying enrollment criteria
                enhanced_data.append([current_id, start_date, current_date, counter])
            
            #reseting start date
            start_date = date_i
            
            #reseting counter
            counter = 1
        
        #prepare date and id for next iteration
        current_date = date_i
        current_id = idi
        
    return enhanced_data

def customize_data(data, time_in_months):
    
    dat = enrollement_check(data, time_in_months)
    
    columns = ['PERS_ID', 'FIRST_DATE', 'LAST_DATE', 'MONTH_COUNTER']
    
    return pd.DataFrame(dat, columns = columns)
    
    
    
    

In [15]:
%%time
# sort pmpm by ids and dates for the enrollment algorithm to work
pmpm.sort_values(by = ['PERS_ID', 'MYR'], inplace = True)
pmpm.reset_index(drop = True, inplace = True)
pmpm_test = pmpm[['PERS_ID', 'MYR']].values

#### find ids with contiuous enrollement up to 12 months
months_of_enrollement = 12
dat = customize_data(pmpm_test, months_of_enrollement)

CPU times: user 9.82 s, sys: 1.02 s, total: 10.8 s
Wall time: 10.8 s


In [16]:
dat.PERS_ID.unique().shape, pmpm.PERS_ID.unique().shape

((90748,), (152408,))

#### Keep only ids that satisfy the enrollement criteria

In [17]:
%%time
unique_ids = dat.PERS_ID.unique()
pmpm = pmpm[pmpm.PERS_ID.isin(unique_ids)]
print(pmpm.PERS_ID.unique().shape)

(90748,)
CPU times: user 497 ms, sys: 3.47 ms, total: 501 ms
Wall time: 500 ms


In [18]:
pmpm.shape, biometrics.shape, pmpm.PERS_ID.unique().shape, biometrics.PERS_ID.unique().shape

((2595031, 74), (3050800, 5), (90748,), (248608,))

#### At this Point we have a Processed Biometrics data file and a fitered PMPM file by Enrollement Criteria
We can merge the two of them on ids and dates

In [19]:
%%time
merged = pmpm.merge(biometrics, left_on = ['PERS_ID', 'MYR'], right_on = ['PERS_ID', 'EVENT_DATE_MYR'])
merged.sort_values(by = ['PERS_ID', 'MYR'], inplace = True, ignore_index = True)

CPU times: user 1.59 s, sys: 383 ms, total: 1.98 s
Wall time: 1.98 s


In [20]:
merged.head(3)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2562798              104        377        0             0   
1  2562798              104        362        0             0   
2  2562798              104        842        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             1   
2                  0       0          0                    0             1   

   ...  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0  ...          0           0                   0                 0         0   
1  ...          0           0                   0                 0         0   
2  ...          0           0                   0                 0         0   

      MYR  EVENT_DATE  BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE  
0  201805  2018-05-01                 BMI          201805     24.930000  
1  201806  2018-06-01                 BMI          201806     24.480000  
2  201807  2018-07-01                 BMI          201807     24.120001  

[3 rows x 78 columns]

<b>If we try to enforce both pmpm and biometric continuous measurements (for example bmi) <br>
we end up with 273 IDS, so we will go by considering measurements <br>
close to refernce dates (that we will compute later)

In [21]:
dat2 = customize_data(merged[['PERS_ID', 'MYR']].values, 12)
dat2.PERS_ID.unique().shape

(273,)

In [22]:
merged.head(3)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2562798              104        377        0             0   
1  2562798              104        362        0             0   
2  2562798              104        842        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             1   
2                  0       0          0                    0             1   

   ...  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0  ...          0           0                   0                 0         0   
1  ...          0           0                   0                 0         0   
2  ...          0           0                   0                 0         0   

      MYR  EVENT_DATE  BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE  
0  201805  2018-05-01                 BMI          201805     24.930000  
1  201806  2018-06-01                 BMI          201806     24.480000  
2  201807  2018-07-01                 BMI          201807     24.120001  

[3 rows x 78 columns]

In [23]:
merged.shape

(440621, 78)

In [24]:
merged.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2562798              104        377        0             0   
1  2562798              104        362        0             0   
2  2562798              104        842        0             0   
3  2562798              104        313        0             0   
4  2562798              104        359        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             1   
2                  0       0          0                    0             1   
3                  0       0          0                    0             1   
4                  0       0          0                    0             1   

   ...  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0  ...          0           0                   0                 0         0   
1  ...          0           0                   0                 0         0   
2  ...          0           0                   0                 0         0   
3  ...          0           0                   0                 0         0   
4  ...          0           0                   0                 0         0   

      MYR  EVENT_DATE  BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE  
0  201805  2018-05-01                 BMI          201805     24.930000  
1  201806  2018-06-01                 BMI          201806     24.480000  
2  201807  2018-07-01                 BMI          201807     24.120001  
3  201809  2018-09-01                 BMI          201809     24.838528  
4  201810  2018-10-01                 BMI          201810     25.560000  

[5 rows x 78 columns]

#### Canary Processing
1. Technical Process (format columns in appropriate form, ids dates etc)
2. Filtering, keep only ids for which we have data


In [25]:
#drop extra columns
canary.drop(labels = ['MBR_ID', 'GROUPNUMBER', 'GROUPNAME', 'LOB', 'FILE_NAME', 'FILE_DATE', 'N_MILESTONES_ACHIEVED', 
                      'LAST_COMPLETED_MILESTONE'], axis = 1, inplace = True)

canary.PERS_ID = canary.PERS_ID.astype(int)

#### Fix dates format make MYR format

In [26]:
canary.REGISTER_DATE = canary.REGISTER_DATE.apply(make_days1)
canary.LAST_ACTIVITY_DATE = canary.LAST_ACTIVITY_DATE.apply(make_days1)
canary['REGISTER_DATE_MYR'] = canary.REGISTER_DATE.apply(make_date_to_MYR)
canary['LAST_ACTIVITY_MYR'] = canary.LAST_ACTIVITY_DATE.apply(make_date_to_MYR)


#### Filter canary data with ids only in merged pmpm - biometrics

In [27]:
canary = canary[canary.PERS_ID.isin(merged.PERS_ID.unique())]
canary.shape

(918, 5)

#### For non Treated the Outcomes will be taken Sequentially For Treated the Outcomes will be taken close to registartion and and date

In [28]:
merged_ids = merged.PERS_ID.unique()
canary_ids = canary.PERS_ID.unique()

control_ids = np.setdiff1d(merged_ids, canary_ids)
print(len(control_ids), len(merged_ids), len(canary_ids))

66439 67357 918


In [29]:
filt_cont = merged.PERS_ID.isin(control_ids)
controls = merged[filt_cont].reset_index(drop = True)
treated  = merged[~filt_cont].reset_index(drop = True)

In [30]:
controls.PERS_ID.unique().shape, treated.PERS_ID.unique().shape

((66439,), (918,))

#### Calculate the sequential outcomes of controls and at the end drop the last month of each Id (because there is no further data for the outcome)

In [31]:
res_value = controls.RESULT_VALUE.values
outcomes = ((res_value[1:]-res_value[0:-1]) < 0).astype(int)
outcome2 = ((res_value[1:]-res_value[0:-1])/res_value[0:-1])

controls = controls.iloc[0:-1]
controls['OUTCOME'] = outcomes
controls['OUTCOME2'] = outcome2


controls = controls.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)
controls.shape[0] 

433774

In [32]:
controls.head(5)

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2562798              104        377        0             0   
1  2562798              104        362        0             0   
2  2562798              104        842        0             0   
3  2562798              104        313        0             0   
4  2562798              104        359        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             1   
2                  0       0          0                    0             1   
3                  0       0          0                    0             1   
4                  0       0          0                    0             1   

   ...  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR     MYR  EVENT_DATE  \
0  ...                   0                 0         0  201805  2018-05-01   
1  ...                   0                 0         0  201806  2018-06-01   
2  ...                   0                 0         0  201807  2018-07-01   
3  ...                   0                 0         0  201809  2018-09-01   
4  ...                   0                 0         0  201810  2018-10-01   

   BIOMETRIC_CATEGORY  EVENT_DATE_MYR  RESULT_VALUE  OUTCOME  OUTCOME2  
0                 BMI          201805     24.930000        1 -0.018051  
1                 BMI          201806     24.480000        1 -0.014706  
2                 BMI          201807     24.120001        0  0.029790  
3                 BMI          201809     24.838528        0  0.029047  
4                 BMI          201810     25.560000        0  0.007042  

[5 rows x 80 columns]

In [33]:
def drop_last(group):
    return group.iloc[0:-1]

In [34]:
%%time
final_dates = controls.groupby('PERS_ID')['MYR'].apply(drop_last).droplevel(1).reset_index()
controls_final = pd.merge(controls, final_dates, on = ['PERS_ID', 'MYR'], how = 'inner')
controls_final['TREATED'] = 0

CPU times: user 20 s, sys: 190 ms, total: 20.2 s
Wall time: 20.1 s


#### Keep only Measurements that were taken at least 120 days apart 

In [35]:
days = 120
date_diff = pd.Series(controls.EVENT_DATE.iloc[1:].values - controls.EVENT_DATE.iloc[0: -1].values).dt.days
time_filt = date_diff >= days

#remove outcomes that do not satisfy the time filter
controls = controls.iloc[0:-1][time_filt].reset_index(drop = True)


In [36]:
time_filt.sum(), date_diff.shape

(139215, (433773,))

In [37]:
controls_final.head()

PERS_ID  AGE_AT_MIDMONTH  ALLOW_AMT  CC_ADHD  CC_ALZHEIMER  \
0  2562798              104        377        0             0   
1  2562798              104        362        0             0   
2  2562798              104        842        0             0   
3  2562798              104        313        0             0   
4  2562798              104        359        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    0             1   
1                  0       0          0                    0             1   
2                  0       0          0                    0             1   
3                  0       0          0                    0             1   
4                  0       0          0                    0             1   

   ...  CC_VISUAL_IMPAIR  MBR_GNDR     MYR  EVENT_DATE  BIOMETRIC_CATEGORY  \
0  ...                 0         0  201805  2018-05-01                 BMI   
1  ...                 0         0  201806  2018-06-01                 BMI   
2  ...                 0         0  201807  2018-07-01                 BMI   
3  ...                 0         0  201809  2018-09-01                 BMI   
4  ...                 0         0  201810  2018-10-01                 BMI   

   EVENT_DATE_MYR  RESULT_VALUE  OUTCOME  OUTCOME2  TREATED  
0          201805     24.930000        1 -0.018051        0  
1          201806     24.480000        1 -0.014706        0  
2          201807     24.120001        0  0.029790        0  
3          201809     24.838528        0  0.029047        0  
4          201810     25.560000        0  0.007042        0  

[5 rows x 81 columns]

#### Now find the outcomes for treated data

In [38]:
treated = treated.drop_duplicates(subset = ['PERS_ID', 'MYR']).reset_index(drop = True)
treated2 = pd.merge(treated, canary, on = 'PERS_ID')
treated2.shape

treated2['REF_START']  = np.abs(treated2.EVENT_DATE - treated2.REGISTER_DATE)
treated2['REF_END']  = np.abs(treated2.EVENT_DATE - treated2.LAST_ACTIVITY_DATE)

In [39]:
def pick_covariates( group ):
    """
    choses rows closer 
    to registration and last activity date
    
    """
    indx_start = np.argmin(group['REF_START'].values)
    indx_end   = np.argmin(group['REF_END'].values)
    N = len(group) - 1
    if indx_start == indx_end:
        if indx_start != N:
            indx_end = N
        else:
            indx_start = 0
            
    data = group.iloc[[indx_start, indx_end]]    
    data['FF'] = 0
    data.iloc[1,-1] = 1
    data['OUTCOME'] = int((data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])<0)
    data['OUTCOME2']  = (data['RESULT_VALUE'].iloc[-1] - data['RESULT_VALUE'].iloc[0])/data['RESULT_VALUE'].iloc[0]
    return data
        
        
    

In [40]:
%%time
treated3 = treated2.groupby(by = 'PERS_ID').apply(pick_covariates)
treated3.index = range(len(treated3))
treated3 = treated3[treated3.FF == 0].reset_index(drop = True)
treated3['TREATED'] = 1

CPU times: user 3.45 s, sys: 0 ns, total: 3.45 s
Wall time: 3.44 s


In [41]:
common_columns = np.intersect1d(controls_final.columns, treated3.columns)

In [42]:
common_columns

array(['AGE_AT_MIDMONTH', 'ALLOW_AMT', 'BIOMETRIC_CATEGORY', 'CC_ADHD',
       'CC_ALZHEIMER', 'CC_ALZHEIMER_PLUS', 'CC_AMI', 'CC_ANEMIA',
       'CC_ANXIETY_DISORDER', 'CC_ARTHRITIS', 'CC_ASTHMA',
       'CC_ATRIAL_FIB', 'CC_AUTISM', 'CC_BIPOLAR', 'CC_BPH',
       'CC_BREAST_CANCER', 'CC_CATARACT', 'CC_CEREBRAL_PALSY',
       'CC_CHRONIC_KIDNEY', 'CC_COLORECTAL_CANCER', 'CC_COPD',
       'CC_CYSTIC_FIBROSIS', 'CC_DEPRESSION', 'CC_DEPRESSIVE_DISORDERS',
       'CC_DEVELOP_DELAYS', 'CC_DIABETES', 'CC_DISAB_DX_CNT', 'CC_DX_CNT',
       'CC_ENDOMETRIAL_CANCER', 'CC_EPILEPSY', 'CC_FIBROMYALGIA',
       'CC_GLAUCOMA', 'CC_HEARING_IMPAIR', 'CC_HEART_FAILURE',
       'CC_HEPATITIS_A', 'CC_HEPATITIS_B_ACUTE', 'CC_HEPATITIS_B_CHRONIC',
       'CC_HEPATITIS_C_ACUTE', 'CC_HEPATITIS_C_CHRONIC',
       'CC_HEPATITIS_C_UNSPECIFIED', 'CC_HEPATITIS_D', 'CC_HEPATITIS_E',
       'CC_HEPATITIS_GEN', 'CC_HIP_FRACTURE', 'CC_HYPERLIPIDEMIA',
       'CC_HYPERTENSION', 'CC_HYPOTHYROID', 'CC_INT_DISAB', 'CC_IS

In [43]:
treated_final = treated3[common_columns]
treated_final.shape

(918, 81)

#### Concatenate controls and treated

In [44]:
df = pd.concat((treated_final, controls_final), axis = 0, ignore_index = True)
df.shape, treated_final.shape, controls_final.shape

((368253, 81), (918, 81), (367335, 81))

#### drop extra columns

In [45]:
to_drop = ['EVENT_DATE', 'EVENT_DATE_MYR', 'MYR']
df = df.drop(labels = to_drop, axis = 1)

In [46]:
df.head()

AGE_AT_MIDMONTH  ALLOW_AMT BIOMETRIC_CATEGORY  CC_ADHD  CC_ALZHEIMER  \
0               81        414                BMI        0             0   
1               82        101                BMI        0             0   
2               78      13007                BMI        0             0   
3               77          0                BMI        0             0   
4               76        898                BMI        0             0   

   CC_ALZHEIMER_PLUS  CC_AMI  CC_ANEMIA  CC_ANXIETY_DISORDER  CC_ARTHRITIS  \
0                  0       0          0                    1             0   
1                  0       0          0                    1             0   
2                  0       0          0                    0             1   
3                  0       0          0                    0             0   
4                  0       0          0                    0             1   

   ...  CC_STROKE  CC_TOBACCO  CC_TRAUMATIC_BRAIN  CC_VISUAL_IMPAIR  MBR_GNDR  \
0  ...          0           0                   0                 0         0   
1  ...          0           0                   0                 0         0   
2  ...          0           0                   0                 0         1   
3  ...          0           0                   0                 0         0   
4  ...          0           0                   0                 0         1   

   OUTCOME  OUTCOME2  PERS_ID  RESULT_VALUE  TREATED  
0        1 -0.020542  2602482     27.772373        1  
1        0  0.007214  2603072     29.256837        1  
2        0  0.094824  2621187     23.289228        1  
3        0  0.040909  2623521     33.928182        1  
4        1 -0.340599  2633716     40.305640        1  

[5 rows x 78 columns]

In [47]:
df.TREATED.value_counts()

0    367335
1       918
Name: TREATED, dtype: int64

In [48]:
Path('../../matching_data').mkdir(parents = True, exist_ok = True)
df.to_pickle('../../matching_data/data.p')